In [22]:
import numpy as np
import os
from PIL import Image
from scipy.ndimage.morphology import binary_dilation
from scipy.ndimage import generate_binary_structure

In [5]:
# Required input for flow_rgb overlay: 
#  * flow as .png
#  * rgb as .jpg
flow_file = '/usr/stud/wangyu/flow-code-python/flow00000.png'
rgb_file = '/usr/stud/wangyu/flow-code-python/rgb00000.jpg'

flow_img = Image.open(flow_file)
rgb_img = Image.open(rgb_file)
com_flow_img = Image.blend(rgb_img, flow_img, 0.6)
#com_flow_img.save('flow_rgb00000.jpg')

In [6]:
# Required input for seg_rgb overlay:
#  * seg as .png
#  * rgb as .jpg
seg_file = '/usr/stud/wangyu/flow-code-python/seg00000.png'
rgb_file = '/usr/stud/wangyu/flow-code-python/rgb00000.jpg'
img_palette = np.array([0,0,0,150,0,0])
rgb_img = Image.open(rgb_file)
seg_arr = np.array(Image.open(seg_file), dtype=np.uint8)
seg_bool = np.greater(seg_arr, 0)
seg_arr = seg_bool.astype(np.uint8)
seg_color = Image.fromarray(seg_arr, mode='P')
seg_color.putpalette(img_palette)
overlay_mask = Image.fromarray(seg_arr*150, mode='L')

com_img = Image.composite(seg_color, rgb_img, overlay_mask)
#com_img.show()
#com_img.save('seg_rgb00000.jpg')

In [24]:
### Generate attention area from [pure optical flow, segmetation]
# Required input:
#  * flow as .flo
#  * seg  as .png
def seg_rgb_overlay(rgb_file, seg_file):
    
    img_palette = np.array([0,0,0,150,0,0])
    rgb_img = Image.open(rgb_file)
    seg_arr = np.array(Image.open(seg_file), dtype=np.uint8)
    seg_bool = np.greater(seg_arr, 0)
    seg_arr = seg_bool.astype(np.uint8)
    seg_color = Image.fromarray(seg_arr, mode='P')
    seg_color.putpalette(img_palette)
    overlay_mask = Image.fromarray(seg_arr*150, mode='L')

    com_img = Image.composite(seg_color, rgb_img, overlay_mask)
        
    return com_img

TAG_FLOAT = 202021.25
def read(file):

    assert type(file) is str, "file is not str %r" % str(file)
    assert os.path.isfile(file) is True, "file does not exist %r" % str(file)
    assert file[-4:] == '.flo', "file ending is not .flo %r" % file[-4:]
    f = open(file,'rb')
    flo_number = np.fromfile(f, np.float32, count=1)[0]
    assert flo_number == TAG_FLOAT, 'Flow number %r incorrect. Invalid .flo file' % flo_number
    w = np.fromfile(f, np.int32, count=1)
    h = np.fromfile(f, np.int32, count=1)
    #if error try: data = np.fromfile(f, np.float32, count=2*w[0]*h[0])
    data = np.fromfile(f, np.float32, count=2*w*h)
    # Reshape data into 3D array (columns, rows, bands)
    flow = np.resize(data, (int(h), int(w), 2))	
    f.close()

    return flow

def transf_slow(seg_arr, flow_arr):
    
    seg_shape = seg_arr.shape
    flow_shape = flow_arr.shape
    assert seg_shape[0] == flow_shape[0], "Seg/Flow shape does not match!"
    assert seg_shape[1] == flow_shape[1], "Seg/Flow shape does not match!"
    h = seg_shape[0]
    w = seg_shape[1]
    new_seg = np.zeros((h, w), np.uint8)
    for idx_h in range(h):
        for idx_w in range(w):
            if seg_arr[idx_h][idx_w] == 1:
                motion_h = int(flow_arr[idx_h][idx_w][0])
                motion_w = int(flow_arr[idx_h][idx_w][1])
                new_h = idx_h + motion_h
                new_w = idx_w + motion_w
                if new_h < h and new_h >= 0 and new_w < w and new_w >= 0:
                    new_seg[new_h][new_w] = 1
    return new_seg

def transf_fast(seg_arr, flow_arr):
    pass
    return

flow_file = '00000.flo'
seg_file = '/usr/stud/wangyu/flow-code-python/seg00000.png'
seg_arr = np.array(Image.open(seg_file), dtype=np.uint8)
seg_arr = np.greater(seg_arr, 0).astype(np.uint8)

# Show [flow, original seg] overlaid
img_palette = np.array([0,0,0,150,0,0])
seg_color = Image.fromarray(seg_arr, mode='P')
seg_color.putpalette(img_palette)
overlay_mask = Image.fromarray(seg_arr*150, mode='L')
com_img = Image.composite(seg_color, flow_img, overlay_mask)
#com_img.show()

# Show flow transformed seg and attention area
flow_arr = read(flow_file)
new_seg_arr = transf_slow(seg_arr, flow_arr)
struct1 = generate_binary_structure(2, 2)
att_flow = binary_dilation(new_seg_arr, structure=struct1, iterations=30).astype(new_seg_arr.dtype)
new_seg_color = Image.fromarray(new_seg_arr, mode='P')
new_seg_color.putpalette(img_palette)
att_flow_color = Image.fromarray(att_flow, mode='P')
att_flow_color.putpalette(img_palette)
#new_seg_color.show()
new_seg_color.save('flow_warped_seg00001.png')
att_flow_color.save('flow_warped_att00001.png')
com_flow_rgb = seg_rgb_overlay('/usr/stud/wangyu/flow-code-python/rgb00001.jpg', 
                               'flow_warped_seg00001.png')
com_att_rgb = seg_rgb_overlay('/usr/stud/wangyu/flow-code-python/rgb00001.jpg', 
                              'flow_warped_att00001.png')
com_flow_rgb.save('flow_seg_rgb00001.jpg')
com_att_rgb.save('flow_att_rgb00001.jpg')